In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [2]:
import os
os.chdir('D:\\R Studio')

In [3]:
data = pd.read_csv("NSSO68.csv")

C:\Users\SAMPREETH\AppData\Local\Temp\ipykernel_18668\4031375496.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("NSSO68.csv")


In [5]:
data['state_1'].unique()

array(['GUJ', 'ORI', 'CHTSD', 'MP', 'JRKD', 'WB', 'AP', 'MH', 'D&D',
       'D&NH', 'MIZ', 'TRPR', 'MANPR', 'ASSM', 'MEG', 'NAG', 'A&N',
       'PNDCRY', 'TN', 'GOA', 'KA', 'KE', 'LKSDP', 'SKM', 'Bhr', 'UP',
       'RJ', 'ARP', 'DL', 'HR', 'Pun', 'HP', 'UT', 'Chandr', 'J$K'],
      dtype=object)

In [45]:
#Subsetting the data
subset_data = data[data['state_1'] == 'KA'][['foodtotal_q', 'MPCE_MRP', 'MPCE_URP', 'Age', 'Meals_At_Home', 'Possess_ration_card', 'Education', 'No_of_Meals_per_day']]
print(subset_data)

       foodtotal_q  MPCE_MRP  MPCE_URP  Age  Meals_At_Home  \
23109    17.925351   1124.92    982.00   38           54.0   
23110    25.517352   2336.88   1704.67   45           30.0   
23111    25.470640   1660.76   1243.80   88           60.0   
23112    29.984010   2457.52   1853.00   63           60.0   
23113    17.760602   1334.57   1221.80   33           58.0   
...            ...       ...       ...  ...            ...   
64086    16.200258    789.43    738.33   46           90.0   
64087    24.787846   1589.75   1250.50   58           90.0   
64088    15.100283    848.95    719.50   37           90.0   
64089    16.175296    824.82    766.88   40           90.0   
64090    23.075418   1370.72   1212.50   30           90.0   

       Possess_ration_card  Education  No_of_Meals_per_day  
23109                  1.0        6.0                  2.0  
23110                  2.0        7.0                  2.0  
23111                  1.0        6.0                  2.0  
23112      

In [8]:
#Checking for missing values
print(subset_data['MPCE_MRP'].isna().sum())
print(subset_data['MPCE_URP'].isna().sum())
print(subset_data['Age'].isna().sum())
print(subset_data['Possess_ration_card'].isna().sum())
print(data['Education'].isna().sum())

0
0
0
0
7


In [46]:
#Creating a function to impute th emissing values with the mean of the variable
def impute_with_mean(data, columns):
    for column in columns:
        data[column].fillna(data[column].mean(), inplace=True)
    return data

In [47]:
#Imputiong the columns
columns_to_impute = ['Education', 'MPCE_MRP', 'MPCE_URP', 'Age', 'Meals_At_Home', 'Possess_ration_card', 'foodtotal_q']

In [48]:
subset_data = impute_with_mean(subset_data, columns_to_impute)

In [49]:
print(subset_data.isna().sum()) 

foodtotal_q            0
MPCE_MRP               0
MPCE_URP               0
Age                    0
Meals_At_Home          0
Possess_ration_card    0
Education              0
No_of_Meals_per_day    0
dtype: int64


In [51]:
#Fitting the regression model
X = subset_data[['MPCE_MRP', 'MPCE_URP', 'Age', 'Meals_At_Home', 'Possess_ration_card', 'Education']]
X = sm.add_constant(X)  # Adds a constant term to the predictor
y = subset_data['foodtotal_q']


In [53]:
model = sm.OLS(y, X).fit()

In [54]:
#Printinf the regression results
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:            foodtotal_q   R-squared:                       0.171
Model:                            OLS   Adj. R-squared:                  0.170
Method:                 Least Squares   F-statistic:                     140.6
Date:                Sun, 23 Jun 2024   Prob (F-statistic):          1.66e-162
Time:                        13:57:54   Log-Likelihood:                -14354.
No. Observations:                4094   AIC:                         2.872e+04
Df Residuals:                    4087   BIC:                         2.877e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                  12.0534    

In [55]:
#Checking for multicollinearity using Inflator Factor (VIF)
vif_data = pd.DataFrame()
vif_data['feature'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]


In [56]:
print(vif_data)

               feature        VIF
0                const  47.066548
1             MPCE_MRP   1.681909
2             MPCE_URP   1.512695
3                  Age   1.117918
4        Meals_At_Home   1.107049
5  Possess_ration_card   1.171771
6            Education   1.222947


In [58]:
#Extracting the coefficients from the model
coefficients = model.params

In [59]:
#Constructing the equation
equation = f"y = {coefficients[0]:.2f}"
for i in range(1, len(coefficients)):
    equation += f" + {coefficients[i]:.6f}*x{i}"
print(equation)


y = 12.05 + 0.000941*x1 + 0.000095*x2 + 0.129580*x3 + 0.037396*x4 + -2.993684*x5 + 0.247028*x6
